In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
import optuna

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import warnings
warnings.filterwarnings('ignore')

# Configuration

In [2]:
ROOT = ".."
INPUT = "input"
LISH_MOA = "lish-moa"
NUM_FOLD = 5
NUM_OPTUNA_TRIAL = 30

# Read data

In [3]:
train = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "train_features.csv"))
test = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "test_features.csv"))
train_targets_scored = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "train_targets_scored.csv"))
train_targets_nonscored = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "train_targets_nonscored.csv"))
sub = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "sample_submission.csv"))

In [4]:
train_targets_scored.columns

Index(['sig_id', '5-alpha_reductase_inhibitor', '11-beta-hsd1_inhibitor',
       'acat_inhibitor', 'acetylcholine_receptor_agonist',
       'acetylcholine_receptor_antagonist', 'acetylcholinesterase_inhibitor',
       'adenosine_receptor_agonist', 'adenosine_receptor_antagonist',
       'adenylyl_cyclase_activator',
       ...
       'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist',
       'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor',
       'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b',
       'vitamin_d_receptor_agonist', 'wnt_inhibitor'],
      dtype='object', length=207)

In [5]:
for col in train_targets_scored.columns:
    if col != 'sig_id':
        c = train_targets_scored[col].value_counts()
        if c[1] <= NUM_FOLD:
            print(col)

atp-sensitive_potassium_channel_antagonist
erbb2_inhibitor


## train_features.csv

In [6]:
print(train.shape)

(23814, 876)


In [7]:
train.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


## train_targets_socred.csv

In [8]:
print(train_targets_scored.shape)

(23814, 207)


In [9]:
train_targets_scored.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## test_features.csv

In [10]:
print(test.shape)

(3982, 876)


In [11]:
test.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380


## train_targets_nonscored.csv (not use)

In [12]:
print(train_targets_nonscored.shape)

(23814, 403)


In [13]:
train_targets_nonscored.head()

,sig_id,abc_transporter_expression_enhancer,abl_inhibitor,ace_inhibitor,acetylcholine_release_enhancer,adenosine_deaminase_inhibitor,adenosine_kinase_inhibitor,adenylyl_cyclase_inhibitor,age_inhibitor,alcohol_dehydrogenase_inhibitor,...,ve-cadherin_antagonist,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## submission.csv

In [14]:
sub.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,id_001897cda,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2,id_002429b5b,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3,id_00276f245,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,id_0027f1083,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


# Preprocessing

## Label Encoding

In [15]:
def label_encoding(train: pd.DataFrame, test: pd.DataFrame, encode_cols):
    n_train = len(train)
    train = pd.concat([train, test], sort=False).reset_index(drop=True)
    for f in encode_cols:
        try:
            lbl = preprocessing.LabelEncoder()
            train[f] = lbl.fit_transform(list(train[f].values))
        except:
            print(f)
    test = train[n_train:].reset_index(drop=True)
    train = train[:n_train]
    return train, test

In [16]:
train.select_dtypes(include=['object']).columns

Index(['sig_id', 'cp_type', 'cp_dose'], dtype='object')

In [17]:
# train['cp_type'] = train['cp_type'].astype(str)
# train['cp_dose'] = train['cp_dose'].astype(str)
# test['cp_type'] = test['cp_type'].astype(str)
# test['cp_dose'] = test['cp_dose'].astype(str)

In [18]:
train, test = label_encoding(train, test, ['cp_type', 'cp_dose'])

## Standrization

In [19]:
tr_mean = train.iloc[:, 1:].mean()
tr_std = train.iloc[:, 1:].std()
train.iloc[:, 1:] = (train.iloc[:, 1:] - tr_mean) / tr_std
test.iloc[:, 1:] = (test.iloc[:, 1:] - tr_mean) / tr_std

In [20]:
train.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,0.291574,-1.237973,-0.980022,0.583920,0.804300,-0.386349,-0.739750,-0.243919,-0.740354,...,0.377630,0.352421,0.659863,0.499473,0.146275,0.621206,0.031745,0.347579,0.463451,0.510758
1,id_000779bfc,0.291574,1.235896,-0.980022,-0.124922,0.620885,0.141781,-0.022706,0.931752,0.559218,...,0.021367,0.595214,0.491607,0.248275,0.371734,0.517787,0.298916,0.294868,0.587495,0.737689
2,id_000a6266a,0.291574,-0.001039,-0.980022,0.272452,0.833843,1.353388,-0.166705,-0.086859,1.168263,...,-0.127847,-0.082395,0.561297,0.247943,-0.378270,0.022295,-0.086524,0.093649,-0.511082,0.706437
3,id_0015fd391,0.291574,-0.001039,-0.980022,-0.546984,-0.188852,-0.403438,0.470340,3.880135,-0.568655,...,-0.814679,-0.089445,-2.556182,-0.416618,-0.164896,-0.573424,-0.561976,-0.293587,0.045046,-0.365003
4,id_001626bd3,0.291574,1.235896,1.020342,-0.411775,-0.375714,0.789535,0.642022,1.318346,-0.581288,...,0.236664,0.228258,0.589623,0.744691,0.490553,0.031072,0.278137,0.391369,0.049951,0.720216


In [21]:
test.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,0.291574,-1.237973,-0.980022,-0.569949,0.278550,-0.642786,0.377709,1.446241,-0.021676,...,0.283603,0.616512,0.184970,0.139263,0.128169,0.143882,0.235218,0.192556,0.338589,-0.336309
1,id_001897cda,0.291574,1.235896,-0.980022,-0.309507,0.403371,1.019325,-0.562279,-0.409408,-0.163699,...,0.175079,0.135233,-0.258650,-0.411397,0.063944,-0.113224,0.242452,-0.057618,-0.496367,-0.304346
2,id_002429b5b,-3.429514,-1.237973,-0.980022,-0.045333,-0.055044,-0.524608,0.051608,-1.448851,0.325877,...,0.121542,0.390903,-0.434999,0.645693,-0.669529,-0.405253,0.645564,0.818402,0.524655,0.076216
3,id_00276f245,0.291574,-1.237973,1.020342,0.168246,0.358440,0.222303,0.360447,-0.622858,-0.901454,...,0.297549,0.302775,0.167285,-0.294175,0.034540,-0.335590,0.619784,0.145421,0.563950,-0.198445
4,id_0027f1083,0.291574,-0.001039,-0.980022,-0.463806,-1.443093,1.700003,0.130239,-0.623731,0.103643,...,0.482754,0.596928,0.177476,0.736149,0.500694,0.886107,0.081217,-0.089843,0.797542,1.519624


# Cross Validation by using Optuna

In [22]:
def logistic_elasticnet_cv(data, targets, target_col, cv):
    def objective(trial):
        param = {
            'penalty': 'l1',
            'C': trial.suggest_loguniform('C', 2**(-10), 2**5),
            'solver': 'liblinear',
            'n_jobs': 8
        }
        
        df_x = data.drop(["sig_id"], axis=1)
        df_y = targets[target_col]
        score_logloss = 0.
        
        # cross validation
        for fold_id, (train_index, valid_index) in enumerate(tqdm(cv.split(df_x, df_y))):
            X_train = df_x.loc[train_index, :]
            y_train = df_y[train_index]
            X_valid = df_x.loc[valid_index, :]
            y_valid = df_y.loc[valid_index]
            
            model = LogisticRegression(**param)
            model.fit(X_train, y_train)
            y_pred = model.predict_proba(X_valid)
            score_logloss += log_loss(y_valid, y_pred, labels=[0, 1])

        score_logloss /= NUM_FOLD
        return score_logloss
    return objective
        

In [23]:
cv = StratifiedKFold(n_splits=NUM_FOLD, shuffle=True, random_state=0)

In [24]:
skip_cols = ["atp-sensitive_potassium_channel_antagonist", "erbb2_inhibitor"]

In [25]:
for target_col in tqdm(train_targets_scored.columns[101:]):
    if (target_col != "sig_id") and not (target_col in skip_cols):
        print('##################### CV START: {0} #####################'.format(target_col))
        
        # optimize by optuna
        study = optuna.create_study()
        study.optimize(logistic_elasticnet_cv(train, train_targets_scored, target_col, cv), n_trials=NUM_OPTUNA_TRIAL)
        
        # save optuna log
        df_trial = study.trials_dataframe()
        df_trial.to_pickle(os.path.join('report', 'LR_OptunaLog_'+target_col+'.pkl'))

        # save best params found by CV
        bestparam_filename = 'LR_BestParamsSelectedByCV_'+target_col+'.pkl'
        with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'wb') as f:
            pickle.dump(study.best_params, f)


  0%|                                                                                          | 0/106 [00:00<?, ?it/s]

##################### CV START: gonadotropin_receptor_agonist #####################



0it [00:00, ?it/s]
1it [00:01,  1.79s/it]
2it [00:03,  1.80s/it]
3it [00:05,  1.82s/it]
4it [00:07,  1.83s/it]
5it [00:09,  1.83s/it]
[I 2020-10-21 22:30:36,443] Finished trial#0 with value: 0.007112737650587666 with parameters: {'C': 0.6124815023133238}. Best is trial#0 with value: 0.007112737650587666.

0it [00:00, ?it/s]
1it [00:00,  1.50it/s]
2it [00:01,  1.48it/s]
3it [00:02,  1.46it/s]
4it [00:02,  1.47it/s]
5it [00:03,  1.47it/s]
[I 2020-10-21 22:30:39,965] Finished trial#1 with value: 0.006781633845603044 with parameters: {'C': 0.03819867831509039}. Best is trial#1 with value: 0.006781633845603044.

0it [00:00, ?it/s]
1it [00:02,  2.60s/it]
2it [00:05,  2.63s/it]
3it [00:08,  2.71s/it]
4it [00:10,  2.72s/it]
5it [00:13,  2.75s/it]
[I 2020-10-21 22:30:53,850] Finished trial#2 with value: 0.008096739980668977 with parameters: {'C': 1.7195153942067318}. Best is trial#1 with value: 0.006781633845603044.

0it [00:00, ?it/s]
1it [00:00,  1.64it/s]
2it [00:01,  1.61it/s]
3it [00:01, 

##################### CV START: gsk_inhibitor #####################



0it [00:00, ?it/s]
1it [00:02,  2.10s/it]
2it [00:03,  2.03s/it]
3it [00:05,  1.98s/it]
4it [00:07,  2.03s/it]
5it [00:10,  2.01s/it]
[I 2020-10-21 22:34:17,258] Finished trial#0 with value: 0.011291621563928472 with parameters: {'C': 0.40524827773939964}. Best is trial#0 with value: 0.011291621563928472.

0it [00:00, ?it/s]
1it [00:03,  3.34s/it]
2it [00:06,  3.26s/it]
3it [00:09,  3.22s/it]
4it [00:12,  3.25s/it]
5it [00:16,  3.32s/it]
[I 2020-10-21 22:34:33,956] Finished trial#1 with value: 0.013398387170490963 with parameters: {'C': 1.3818841499607626}. Best is trial#0 with value: 0.011291621563928472.

0it [00:00, ?it/s]
1it [00:00,  1.31it/s]
2it [00:01,  1.29it/s]
3it [00:02,  1.30it/s]
4it [00:03,  1.28it/s]
5it [00:03,  1.29it/s]
[I 2020-10-21 22:34:37,944] Finished trial#2 with value: 0.014863849926129685 with parameters: {'C': 0.025946050035614725}. Best is trial#0 with value: 0.011291621563928472.

0it [00:00, ?it/s]
1it [00:01,  1.73s/it]
2it [00:03,  1.69s/it]
3it [00:04

...


0it [00:00, ?it/s]
1it [00:00,  1.61it/s]
2it [00:01,  1.58it/s]
3it [00:01,  1.58it/s]
4it [00:02,  1.59it/s]
5it [00:03,  1.59it/s]
[I 2020-10-22 10:24:24,415] Finished trial#0 with value: 0.008953983101047962 with parameters: {'C': 0.04209058683951607}. Best is trial#0 with value: 0.008953983101047962.

0it [00:00, ?it/s]
1it [00:02,  2.57s/it]
2it [00:05,  2.55s/it]
3it [00:07,  2.52s/it]
4it [00:10,  2.51s/it]
5it [00:12,  2.52s/it]
[I 2020-10-22 10:24:37,103] Finished trial#1 with value: 0.010567319524717362 with parameters: {'C': 0.994836161027748}. Best is trial#0 with value: 0.008953983101047962.

0it [00:00, ?it/s]
1it [00:04,  4.72s/it]
2it [00:09,  4.61s/it]
3it [00:13,  4.56s/it]
4it [00:17,  4.51s/it]
5it [00:22,  4.57s/it]
[I 2020-10-22 10:25:00,045] Finished trial#2 with value: 0.015482610256543286 with parameters: {'C': 11.122419026374839}. Best is trial#0 with value: 0.008953983101047962.

0it [00:00, ?it/s]
1it [00:04,  4.41s/it]
2it [00:08,  4.29s/it]
3it [00:12,  

##################### CV START: vitamin_d_receptor_agonist #####################



0it [00:00, ?it/s]
1it [00:02,  2.67s/it]
2it [00:05,  2.66s/it]
3it [00:08,  2.71s/it]
4it [00:10,  2.67s/it]
5it [00:13,  2.65s/it]
[I 2020-10-22 10:28:26,723] Finished trial#0 with value: 0.004547530103020989 with parameters: {'C': 6.8747310606438745}. Best is trial#0 with value: 0.004547530103020989.

0it [00:00, ?it/s]
1it [00:02,  2.08s/it]
2it [00:04,  2.08s/it]
3it [00:06,  2.10s/it]
4it [00:08,  2.04s/it]
5it [00:10,  2.05s/it]
[I 2020-10-22 10:28:37,100] Finished trial#1 with value: 0.0041950481994187895 with parameters: {'C': 1.4266334291820761}. Best is trial#1 with value: 0.0041950481994187895.

0it [00:00, ?it/s]
1it [00:02,  2.31s/it]
2it [00:04,  2.34s/it]
3it [00:07,  2.34s/it]
4it [00:09,  2.30s/it]
5it [00:11,  2.29s/it]
[I 2020-10-22 10:28:48,662] Finished trial#2 with value: 0.004245496049348962 with parameters: {'C': 2.2552547462253116}. Best is trial#1 with value: 0.0041950481994187895.

0it [00:00, ?it/s]
1it [00:00,  1.26it/s]
2it [00:01,  1.25it/s]
3it [00:02

##################### CV START: wnt_inhibitor #####################



0it [00:00, ?it/s]
1it [00:00,  1.71it/s]
2it [00:01,  1.70it/s]
3it [00:01,  1.69it/s]
4it [00:02,  1.69it/s]
5it [00:02,  1.70it/s]
[I 2020-10-22 10:32:48,861] Finished trial#0 with value: 0.015315668320237171 with parameters: {'C': 0.006444849389260682}. Best is trial#0 with value: 0.015315668320237171.

0it [00:00, ?it/s]
1it [00:03,  3.92s/it]
2it [00:07,  3.81s/it]
3it [00:11,  3.80s/it]
4it [00:14,  3.68s/it]
5it [00:17,  3.59s/it]
[I 2020-10-22 10:33:06,911] Finished trial#1 with value: 0.014804377211314482 with parameters: {'C': 2.136542287905327}. Best is trial#1 with value: 0.014804377211314482.

0it [00:00, ?it/s]
1it [00:02,  2.22s/it]
2it [00:04,  2.19s/it]
3it [00:06,  2.18s/it]
4it [00:08,  2.15s/it]
5it [00:10,  2.11s/it]
[I 2020-10-22 10:33:17,584] Finished trial#2 with value: 0.012024504352097783 with parameters: {'C': 0.5882142128130992}. Best is trial#2 with value: 0.012024504352097783.

0it [00:00, ?it/s]
1it [00:05,  5.13s/it]
2it [00:09,  4.92s/it]
3it [00:14, 

# Train & Predict by best model

In [29]:
submission = sub.copy()

In [30]:
def logistic_elasticnet_bestparam(data_train, targets_train, data_test, target_col, best_params, submission):
    print('##################### TraingByBestParam START: {0} #####################'.format(target_col))
    
    # prepare data set
    X_train = data_train.drop(["sig_id"], axis=1)
    y_train = targets_train[target_col]
    X_test = data_test.drop(["sig_id"], axis=1)
    
    # train XGBoost by using best_params
    model = LogisticRegression(**best_params)
    model.fit(X_train, y_train)
    
    # predict for test
    submission[target_col] = model.predict_proba(X_test)[:, 1]
    
    # save model for test
    modelfile = 'LR_BestModel4test_'+target_col+'.pkl'
    with open(os.path.join('result', 'best_model', modelfile), 'wb') as f:
        pickle.dump(model, f)
    

In [35]:
for target_col in tqdm(train_targets_scored.columns[101:]):
    if target_col != "sig_id" and not (target_col in skip_cols):
        # read best param selected by CV
        bestparam_filename = 'LR_BestParamsSelectedByCV_'+target_col+'.pkl'
        with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'rb') as f:
            best_params = pickle.load(f)
            
        best_params['penalty'] = 'l1'
        best_params['n_jobs'] = 8
        best_params['solver'] = 'liblinear'
        
        logistic_elasticnet_bestparam(train, train_targets_scored, test, target_col, best_params, submission)
    elif target_col in skip_cols:
        submission[target_col] = 1e-05

  0%|                                                                                          | 0/106 [00:00<?, ?it/s]

##################### TraingByBestParam START: gonadotropin_receptor_agonist #####################


  1%|▊                                                                                 | 1/106 [00:01<02:10,  1.24s/it]

##################### TraingByBestParam START: gsk_inhibitor #####################


  2%|█▌                                                                                | 2/106 [00:03<02:27,  1.42s/it]

...

 98%|██████████████████████████████████████████████████████████████████████████████▍ | 104/106 [02:50<00:03,  1.65s/it]

##################### TraingByBestParam START: vitamin_d_receptor_agonist #####################


 99%|███████████████████████████████████████████████████████████████████████████████▏| 105/106 [02:53<00:01,  1.98s/it]

##################### TraingByBestParam START: wnt_inhibitor #####################


100%|████████████████████████████████████████████████████████████████████████████████| 106/106 [02:54<00:00,  1.64s/it]


In [ ]:
submission.to_csv('submission.csv', index=False)